# Aplikasi Image-Scraper
#### Scrape images from the web

-----

Enter the list of keywords in the cell below


In [ ]:
%%writefile keywords.txt
Rumah adat gadang sumatera
Rumah adat gadang
gadang sumatera 

Overwriting keywords.txt


In [ ]:
# Search engine to use while scraping the images. 
search_engine = "all" # [google, bing, yahoo, duckduckgo, all]

# Number of images per keyword. All images are downloaded when set to None
num_images = None

# Add a prefix and/or suffix to all the keywords
prefix = None
suffix = None

# Threshold for removing similar images
similarity_threshold = 0.98

# Output directory to store the scraped images
out_dir = "images"

-----

In [ ]:
!pip install imagehash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 296 kB 4.8 MB/s 


In [ ]:
import os, platform, sys, argparse, glob
import time
import base64
import hashlib
from io import BytesIO
from html.parser import HTMLParser
from urllib.parse import quote, unquote

from tqdm import tqdm
import urllib3
import PIL.Image as Image

import numpy as np
import imagehash

In [ ]:
import warnings
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [ ]:
!pip install selenium
from selenium import webdriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get update 
!apt install chromium-chromedriver -y

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [22.9 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,094 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/u

In [ ]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
# Instantiate and connect to the chrome driver 
def setup_browser():
	options = webdriver.ChromeOptions()
	options.add_argument('--headless')
	options.add_argument('--no-sandbox')
	options.add_argument('--disable-dev-shm-usage')
	browser = webdriver.Chrome('chromedriver', options=options)

	return browser

In [ ]:
# Get the SHA-256 hash of a file
def sha256(fname, size=4096):
 
	sha256_hash = hashlib.sha256()
	with open(fname, 'rb') as f:
		for byte_block in iter(lambda: f.read(4096), b""):
			sha256_hash.update(byte_block)
		
	return sha256_hash.hexdigest()

# Find difference between files using SHA-256 and remove duplicates
def remove_duplicate_images(directory):
	print("\nChecking for duplicate images by comparing SHA-256 hash")
	flag = False

	file_list = glob.glob(f"{directory}/*.png")

	unique = []
	for file in file_list:
		filehash = sha256(file)

		if filehash not in unique:
			unique.append(filehash)
		else:
			print(f"Removing duplicate image: {file}")
			os.remove(file)	
			flag = True
			
	if flag == False:
		print("No duplicate images found")


# Get a combined perceptual hashs of a image
def get_perceptual_hash(img_path):
	img = Image.open(img_path)

	hashes = [
		imagehash.average_hash,
		imagehash.phash,
		imagehash.dhash,
		imagehash.whash,
	]

	combined_hash = np.array([h(img).hash for h in hashes]).flatten()
	combined_hash = np.where(combined_hash==True, 1, 0)

	return combined_hash

# Compare combined perceptual hashs of two images
def compare_hash(hash1, hash2):
	assert len(hash1) == len(hash2)

	count = 0
	for i in range(len(hash1)):
		if hash1[i] == hash2[i]:
			count +=1

	return count/len(hash1)

# Remove similar images using perceptual hashs
def remove_similar_images(directory, similarity_threshold=0.98):
	print("\nChecking for similar images")
	file_list = glob.glob(f"{directory}/*.png")

	found = False
	unique = []
	for file in file_list:
		filehash = get_perceptual_hash(file)

		flag = False
		for each in unique:
			similarity = compare_hash(each, filehash)

			if similarity >= similarity_threshold:
				flag = True
				found = True
				break

		if flag:
			print(f"Removing similar image: {file}")
			os.remove(file)
		else:
			unique.append(filehash)

	if not found:
		print("No similar images found")

In [ ]:

# Class to extract the value of specific HTML tag attribute
class Extractor(HTMLParser):
  src = []
  tag_attr = None
  def handle_starttag(self, tag, attrs):
    if tag == "img":
      for each in attrs:
        if each[0] == self.tag_attr:
          self.src.append(each[1])

# Create output directory if it does not exist
def create_output_directory(keyword, out_dir=None):
	if out_dir == None:
		os.makedirs(keyword, exist_ok=True)		
	else:
		os.makedirs(out_dir, exist_ok=True)
		os.makedirs(f"{out_dir}/{keyword}", exist_ok=True)

def add_prefix_suffix(keyword, prefix=None, suffix=None):
	if prefix != None:
		keyword = prefix + " " + keyword

	if suffix != None:
		keyword = keyword + " " + suffix
	
	return keyword

def filter_src_format(src_list):
	filtered = []

	for each_src in src_list:
		if ".png" in each_src or ".jpg" in each_src or ".jpeg" in each_src:
			filtered.append(each_src)
		elif "/png" in each_src or "/jpg" in each_src or "/jpeg" in each_src:
			filtered.append(each_src)
		elif "https:" in each_src:
			filtered.append(each_src)
		else:
			continue

	return filtered


def get_img_data(url, src):
	if "https:" in src or "www." in src:
		response = http.request('GET', src)
		img_data = BytesIO(response.data)

	elif src.endswith(".png") or src.endswith(".jpg"):
		base_url = urlparse(url).netloc
		src = base_url + src

		response = http.request('GET', src)
		img_data = BytesIO(response.data)

	else:
		src = src.split(',')[-1]
		img_data = base64.b64decode(src)
		img_data = BytesIO(img_data)

	return img_data

# Search for the specified keyword using the specified search engine, load the url on chrome 
# using chromedriver, extract certain attribute values and then collect the images
def scrape_images_search_engine(keyword, search_engine, output_directory, num_images=None):		
	print(f"\nSearch engine: {search_engine}")

	search_engine_urls = {
		"google" : f"https://www.google.com/search?tbm=isch&q={quote(keyword)}",
		"bing" : f"https://www.bing.com/images/search?q={quote(keyword)}",
		"yahoo" : f"https://images.search.yahoo.com/search/images?p={quote(keyword)}",
		"duckduckgo": f"https://duckduckgo.com/?q={quote(keyword)}&iax=images&ia=images"
	}
	url = search_engine_urls[search_engine]
	print(f"URL: {url}")

	browser.get(url)
	time.sleep(2)

	scroll_count = {"google": 3, "bing": 3, "yahoo": 1, "duckduckgo": 5}
	for _ in range(scroll_count[search_engine]):
		browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
		browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
		time.sleep(2)

	extractor.src = []

	if search_engine == "google":
		extractor.tag_attr = "data-src"
		extractor.feed(browser.page_source)
		
		extractor.tag_attr = "src"
		extractor.feed(browser.page_source)

	elif search_engine == "bing":
		extractor.tag_attr = "src"
		extractor.feed(browser.page_source)

		extractor.src = list(map(lambda x: x.split('?w')[0], extractor.src))
		extractor.src = list(set(extractor.src))

		filtered = []
		for each in extractor.src:
			if "OIP" in each:
				filtered.append(each)

		extractor.src = filtered	
		
	elif search_engine == "yahoo":
		extractor.tag_attr = "src"
		extractor.feed(browser.page_source)

		extractor.src = list(map(lambda x: x.split('&')[0], extractor.src))
		extractor.src = list(set(extractor.src))

	elif search_engine == "duckduckgo":
		extractor.tag_attr = "src"
		extractor.feed(browser.page_source)

		extractor.src = list(map(lambda x: unquote(x.split('?')[-1][2:]), extractor.src))

	extractor.src = filter_src_format(extractor.src)
	len_src = len(extractor.src)
	print(f"Number of images found: {len_src}")

	if num_images != None:
		src_list = extractor.src[:num_images]
	else:
		src_list = extractor.src

	count = 0
	for each_src in tqdm(src_list):
		try:
			img_data = get_img_data(url, each_src)

			image = Image.open(img_data).convert("RGBA")
			image.save(f"{output_directory}/{search_engine[0]}-{count+1}.png")
				
			count+=1

		except:
			print(f"Something went wrong while scraping the image at URL:\n{each_src}")

	if num_images != None:
		print(f"Downloaded {count}/{num_images} images")
	else:
		print(f"Downloaded {count}/{len_src} images")

In [ ]:
browser = setup_browser()
extractor = Extractor()
http = urllib3.PoolManager()

if search_engine not in ['google', 'bing', 'yahoo', 'duckduckgo', 'all']:
  raise Exception("Search engine needs to be one of the following: google, bing, yahoo, duckduckgo or all")

if num_images != None:
  num_images = [num_images // 4 + (1 if x < num_images % 4 else 0)  for x in range (4)]

with open("keywords.txt", 'r') as infile:
  keywords = infile.read().splitlines()

  for each in keywords:
    if each != "":
      print('\n' + '-' * 100)
      
      keyword = add_prefix_suffix(each, prefix=prefix, suffix=suffix).strip()
      print(f"Keyword: {keyword}")

      create_output_directory(keyword, out_dir)
      output_directory = keyword if out_dir == None else f"{out_dir}/{keyword}"

      if search_engine != "all":
        scrape_images_search_engine(keyword=keyword, search_engine=search_engine, output_directory=output_directory, num_images=num_images)
      else:
        if num_images != None:
          for i, each_se in enumerate(['google', 'bing', 'yahoo', 'duckduckgo']):
            scrape_images_search_engine(keyword=keyword, search_engine=each_se, output_directory=output_directory, num_images=num_images[i])
        else:
          for each_se in ['google', 'bing', 'yahoo', 'duckduckgo']:
            scrape_images_search_engine(keyword=keyword, search_engine=each_se, output_directory=output_directory, num_images=num_images)

      remove_duplicate_images(output_directory)
      remove_similar_images(output_directory, similarity_threshold)
      print('-' * 100)
    
      time.sleep(2)

browser.quit()


----------------------------------------------------------------------------------------------------
Keyword: Rumah adat gadang sumatera

Search engine: google
URL: https://www.google.com/search?tbm=isch&q=Rumah%20adat%20gadang%20sumatera
Number of images found: 857


 16%|█▌        | 136/857 [00:05<00:29, 24.60it/s]

Something went wrong while scraping the image at URL:
https://encrypted-tbn2.gstatic.com/faviconV2?url=https://smtp.tni.mil.id&client=VFE&size=16&type=FAVICON&fallback_opts=TYPE,SIZE,URL&nfrp=2


 25%|██▌       | 218/857 [00:08<00:21, 30.08it/s]

Something went wrong while scraping the image at URL:
https://encrypted-tbn2.gstatic.com/faviconV2?url=https://jelajahsumbar.com&client=VFE&size=16&type=FAVICON&fallback_opts=TYPE,SIZE,URL&nfrp=2


...

Downloaded 130/130 images

Search engine: yahoo
URL: https://images.search.yahoo.com/search/images?p=gadang%20sumatera
Number of images found: 91


100%|██████████| 91/91 [00:12<00:00,  7.56it/s]


Downloaded 91/91 images

Search engine: duckduckgo
URL: https://duckduckgo.com/?q=gadang%20sumatera&iax=images&ia=images
Number of images found: 501


100%|██████████| 501/501 [00:58<00:00,  8.54it/s]


Downloaded 501/501 images

Checking for duplicate images by comparing SHA-256 hash
Removing duplicate image: images/gadang sumatera/g-11.png
Removing duplicate image: images/gadang sumatera/y-37.png
Removing duplicate image: images/gadang sumatera/b-69.png
Removing duplicate image: images/gadang sumatera/g-9.png
Removing duplicate image: images/gadang sumatera/g-124.png
Removing duplicate image: images/gadang sumatera/d-138.png
Removing duplicate image: images/gadang sumatera/d-44.png
Removing duplicate image: images/gadang sumatera/g-205.png
Removing duplicate image: images/gadang sumatera/d-38.png
Removing duplicate image: images/gadang sumatera/g-393.png
Removing duplicate image: images/gadang sumatera/g-578.png
Removing duplicate image: images/gadang sumatera/g-507.png
Removing duplicate image: images/gadang sumatera/g-521.png
Removing duplicate image: images/gadang sumatera/d-39.png
Removing duplicate image: images/gadang sumatera/b-65.png
Removing duplicate image: images/gadang s

In [ ]:
!zip images.zip images/*/*
!rm -rf images

  adding: images/gadang sumatera/b-100.png (deflated 0%)
  adding: images/gadang sumatera/b-112.png (deflated 0%)
  adding: images/gadang sumatera/b-117.png (deflated 0%)
  adding: images/gadang sumatera/b-121.png (deflated 0%)
  adding: images/gadang sumatera/b-123.png (deflated 0%)
  adding: images/gadang sumatera/b-125.png (deflated 0%)
  adding: images/gadang sumatera/b-126.png (deflated 0%)
  adding: images/gadang sumatera/b-14.png (deflated 0%)
  adding: images/gadang sumatera/b-15.png (deflated 0%)
  adding: images/gadang sumatera/b-17.png (deflated 0%)
  adding: images/gadang sumatera/b-19.png (deflated 1%)
  adding: images/gadang sumatera/b-1.png (deflated 0%)
  adding: images/gadang sumatera/b-24.png (deflated 0%)
  adding: images/gadang sumatera/b-26.png (deflated 0%)
  adding: images/gadang sumatera/b-27.png (deflated 0%)
  adding: images/gadang sumatera/b-29.png (deflated 0%)
  adding: images/gadang sumatera/b-34.png (deflated 0%)
  adding: images/gadang sumatera/b-35.png 

In [ ]:
from google.colab import files
files.download('images.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# !cp images.zip /path/to/directory/images.zip